In [1]:
import torchaudio

from speechbrain.inference.classifiers import EncoderClassifier
from pathlib import Path
from tqdm import tqdm
import pandas as pd

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [2]:
project_root = Path.cwd().absolute()

data_root = project_root / "data" / "voxceleb"

audio_files = data_root / "dev" / "aac"


meta_data_path = data_root / "vox2_meta.csv"

meta_df = pd.read_csv(meta_data_path, delimiter="\t")

print(f"Number of persons:\t{meta_df.shape[0]}")
meta_df.head()

Number of persons:	6114


,Name,VoxCeleb2 ID,VGGFace2 ID,Gender,Set
0,Aaron_Ashmore,id00012,n000012,m,dev
1,Aaron_Motsoaledi,id00015,n000015,m,dev
2,Aaron_Ramsey,id00016,n000016,m,dev
3,Aaron_Rodgers,id00017,n000017,m,test
4,Aaron_Schock,id00018,n000018,m,dev


In [ ]:
language_id = EncoderClassifier.from_hparams(
    source="speechbrain/lang-id-voxlingua107-ecapa",
    savedir="tmp",
    run_opts={"device": "cuda"},
)

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/lang-id-voxlingua107-ecapa' if not cached


hyperparams.yaml:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/lang-id-voxlingua107-ecapa' if not cached
/home/john/mp/voice-finder/.venv/lib/python3.11/site-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/lang-id-voxlingua107-ecapa' if not cached


embedding_model.ckpt:   0%|          | 0.00/84.5M [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/lang-id-voxlingua107-ecapa' if not cached


classifier.ckpt:   0%|          | 0.00/763k [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/lang-id-voxlingua107-ecapa' if not cached


label_encoder.txt:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, classifier, label_encoder


In [ ]:
speaker_folders = list(audio_files.iterdir())

predictions = []

batch_metadata = []
batch_inputs = []

# The audio data is split usnig the folder structure <speaker>/<video-clip>/<sound-clip>

for speaker_folder in tqdm(speaker_folders, total=len(speaker_folders)):
    for clip_folder in speaker_folder.iterdir():
        for clip in clip_folder.iterdir():

            prediction_raw = language_id.classify_file(str(clip))

            predictions.append(
                {
                    "language": prediction_raw[3][0].split(":")[0],
                    "speaker_id": speaker_folder.name,
                    "clip_id": clip_folder.name,
                    "audio_file": clip.name,
                    "path": str(clip.relative_to(audio_files)),
                }
            )
predictions_df = pd.DataFrame(predictions)

predictions_df.to_csv("audio_clips_meta_data.json")
predictions_df.head()

  0%|                                                                            | 0/5994 [00:02<?, ?it/s]


KeyboardInterrupt: 